In [1]:
from articlecommon import *
import warnings
warnings.filterwarnings("ignore")
from category_encoders import TargetEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SequentialFeatureSelector

# Add Policy_Sales_Channel OHE columns
psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")
counts = X_le["Policy_Sales_Channel"].value_counts()
counts = counts[counts == 1]
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(x), counts.index.tolist()))
psc_ohe = psc_ohe.drop(columns=columns_to_remove)

X_le_psc = pd.concat([X_le, psc_ohe], axis=1)

value_counts = X_le_psc["Policy_Sales_Channel"].value_counts().index
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(value_counts[x]), [*range(147, 86, -1)]))
columns_to_remove += ['Vintage']

# Define the model

quant_trans_uniform = QuantileTransformer(output_distribution='uniform')
lgbm = LGBMClassifier(random_state=42, verbose=-1)

model = make_pipeline(quant_trans_uniform, lgbm)

In [2]:
sfs = SequentialFeatureSelector(model, n_features_to_select='auto', tol=0.0000000001, scoring='roc_auc', cv=10)
sfs.fit(X_le_psc.drop(columns=columns_to_remove), y) # TODO: set all records

SequentialFeatureSelector(cv=10,
                          estimator=Pipeline(steps=[('quantiletransformer',
                                                     QuantileTransformer()),
                                                    ('lgbmclassifier',
                                                     LGBMClassifier(random_state=42,
                                                                    verbose=-1))]),
                          n_features_to_select='auto', scoring='roc_auc',
                          tol=1e-10)

In [3]:
# 100 rekordów - 13.5s
# 200 rekordów - 21.3s
# 300 rekordów - 32.8s
# 1000 rekordów - 1m 24.7s

# Przycięte 60 "głupich" kolumn OHE z tol=0.00001:
# 1000 rekordów - 49s
# 2000 rekordów - 1m 44.8s
# 10k rekorów - 3m 30s
# 20k rekordów 3m 33s
# 40k rekordów 5m 24s

# Przycięte 60 "głupich" kolumn OHE z tol=0.00000001:
# 40k rekordów 5m 26s

# Przycięte 60 "głupich" kolumn OHE z tol=0.00000001 i cv=10:
# 20k rekordów 10m 9s
# 40k rekordów 12m 5s

# estymacja 381k rekordów (czas 40k * 10) = ~120 minut

In [4]:
X_le_psc.drop(columns=columns_to_remove).loc[:, sfs.get_support()]

,Gender,Age,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Policy_Sales_Channel,Policy_Sales_Channel_116.0,Policy_Sales_Channel_131.0
0,1,44,28.0,0,2,1,26.0,0,0
1,1,76,3.0,0,0,0,26.0,0,0
2,1,47,28.0,0,2,1,26.0,0,0
3,1,21,11.0,1,1,0,152.0,0,0
4,0,29,41.0,1,1,0,152.0,0,0
...,...,...,...,...,...,...,...,...,...
381104,1,74,26.0,1,0,0,26.0,0,0
381105,1,30,37.0,1,1,0,152.0,0,0
381106,1,21,30.0,1,1,0,160.0,0,0
381107,0,68,14.0,0,2,1,124.0,0,0


In [5]:
columns_series = pd.Series(X_le_psc.drop(columns=columns_to_remove).loc[:, sfs.get_support()].columns.values)
columns_series.to_csv("results/sfs_columns.csv")

In [6]:
cross_val_summary(model, X_le_psc.drop(columns=columns_to_remove), y)

Cross validation time: 15.964802980422974 seconds
Mean score: 0.8576919569781671


In [7]:
cross_val_summary(model, X_le_psc.drop(columns=columns_to_remove).loc[:, sfs.get_support()], y)

Cross validation time: 5.4100658893585205 seconds
Mean score: 0.8577229501518608
